In [1]:
# Imports
import cv2
import mediapipe as mp
from IPython.display import display, clear_output
import time

In [2]:
# Initialize MediaPipe modules
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

pose = mp_pose.Pose()
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2)

In [3]:
# Pose keypoint connections (custom skeleton)
POSE_CONNECTIONS = [
    (11, 13), (13, 15),     # Left arm
    (12, 14), (14, 16),     # Right arm
    (11, 12),               # Shoulders
    (11, 23), (12, 24),     # Torso sides
    (23, 24),               # Hip line
    (23, 25), (25, 27),     # Left leg
    (24, 26), (26, 28),     # Right leg
    (27, 31), (28, 32),     # Feet
    (0, 1), (1, 2), (2, 3), (3, 7),  # Nose to head
    (0, 4), (4, 5), (5, 6), (6, 8)   # Nose to eye/ear
]


In [4]:
def draw_landmark_lines(image, landmarks, connections, color=(0, 255, 0)):
    h, w, _ = image.shape
    for start_idx, end_idx in connections:
        try:
            x1 = int(landmarks[start_idx].x * w)
            y1 = int(landmarks[start_idx].y * h)
            x2 = int(landmarks[end_idx].x * w)
            y2 = int(landmarks[end_idx].y * h)
            cv2.line(image, (x1, y1), (x2, y2), color, 2)
        except:
            continue


In [5]:
cap = cv2.VideoCapture(0)

In [6]:
print("Press 'q' in the webcam window to quit")

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        print("Camera error!")
        break

    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    pose_result = pose.process(rgb)
    hand_result = hands.process(rgb)

    # Draw pose landmarks manually
    if pose_result.pose_landmarks:
        draw_landmark_lines(frame, pose_result.pose_landmarks.landmark, POSE_CONNECTIONS, color=(0, 255, 0))

    # Draw hands
    if hand_result.multi_hand_landmarks:
        for hand_landmarks in hand_result.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(0, 255, 255), thickness=2)
            )

    # Show frame
    cv2.imshow("Stickman Pose + Hands", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Press 'q' in the webcam window to quit
